Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.2907, -0.2055,  0.2982,  0.3197,  0.1565, -0.1631],
        [ 0.3893,  0.2953,  0.0907,  0.0271,  0.2381, -0.1860],
        [ 0.0051, -0.0883,  0.3482,  0.2405,  0.2506, -0.3392],
        [ 0.2174,  0.2016,  0.0303, -0.3464,  0.4008, -0.2067],
        [ 0.2815, -0.3512,  0.3689, -0.0063,  0.1431, -0.2717],
        [-0.4000,  0.2374,  0.4027, -0.2685, -0.1086, -0.2655],
        [ 0.2590,  0.1994, -0.2735, -0.1729,  0.0325, -0.0700],
        [-0.2275, -0.3165,  0.4048, -0.1405, -0.2994, -0.1076],
        [ 0.3525, -0.1236, -0.1773, -0.2584,  0.2590, -0.2946],
        [-0.0711,  0.2733, -0.3781, -0.1010, -0.2663, -0.1451],
        [-0.0571,  0.2525, -0.2079, -0.2047, -0.0184, -0.0353],
        [-0.2268, -0.2070,  0.3500, -0.3745, -0.1381, -0.3560],
        [ 0.0457,  0.2700,  0.0446,  0.3617, -0.1899, -0.3012],
        [ 0.2909,  0.3535,  0.2051,  0.1199, -0.3351, -0.3806],
        [ 0.1586, -0.2400,  0.3184, -0.3068, -0.1365,  0.0701],
        [ 0.3324, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.1107],
        [-0.1108],
        [-0.1093],
        ...,
        [-0.1121],
        [-0.1090],
        [-0.1103]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



<font color='lightseagreen'> Há dois modos no pytorch, a rede de treino e a rede de avaliação. Explicitamos aqui, que a rede está no modo treino. </font>

In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.1422)
1 tensor(0.1389)
2 tensor(0.1355)
3 tensor(0.1322)
4 tensor(0.1290)
5 tensor(0.1258)
6 tensor(0.1226)
7 tensor(0.1194)
8 tensor(0.1162)
9 tensor(0.1128)
10 tensor(0.1094)
11 tensor(0.1060)
12 tensor(0.1025)
13 tensor(0.0990)
14 tensor(0.0952)
15 tensor(0.0913)
16 tensor(0.0873)
17 tensor(0.0834)
18 tensor(0.0795)
19 tensor(0.0758)
20 tensor(0.0721)
21 tensor(0.0685)
22 tensor(0.0651)
23 tensor(0.0619)
24 tensor(0.0589)
25 tensor(0.0561)
26 tensor(0.0535)
27 tensor(0.0511)
28 tensor(0.0490)
29 tensor(0.0472)
30 tensor(0.0456)
31 tensor(0.0443)
32 tensor(0.0432)
33 tensor(0.0424)
34 tensor(0.0418)
35 tensor(0.0414)
36 tensor(0.0412)
37 tensor(0.0411)
38 tensor(0.0411)
39 tensor(0.0412)
40 tensor(0.0412)
41 tensor(0.0412)
42 tensor(0.0412)
43 tensor(0.0411)
44 tensor(0.0409)
45 tensor(0.0407)
46 tensor(0.0403)
47 tensor(0.0399)
48 tensor(0.0395)
49 tensor(0.0390)
50 tensor(0.0385)
51 tensor(0.0381)
52 tensor(0.0376)
53 tensor(0.0372)
54 tensor(0.0368)
55 tensor(0.0363)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1481.25110483]
[1409.99999118] [1497.43944407]
[1128.99997044] [1317.25652552]
[12991.99967575] [11884.84692001]
[14720.0001564] [14784.85163116]
[2938.99994564] [2642.94345474]
[6635.00014687] [7939.27713203]
[11215.99973297] [9178.622715]
[1040.99999189] [1493.74872541]
[5040.00003624] [5213.5721283]
[9541.99988747] [8897.015728]
[3792.99988461] [5351.84495354]
[1002.0000155] [741.10464096]
[2542.00005341] [1497.66875315]
[6788.00001717] [6690.20300102]
[9386.00025749] [8320.54868126]
[1245.99996853] [1883.84950113]
[997.00000262] [677.73912764]
[708.00000262] [781.40323162]
[1435.99997544] [1428.02232075]
[2590.00006676] [2558.55206251]
[1392.99997497] [1653.62318277]
[11755.99946976] [9539.03485489]
[2007.99997377] [1312.37791967]
[2707.99998474] [2545.74162245]
[631.00000417] [750.82492065]
[652.99999881] [1004.29992771]
[2492.00006247] [2700.4482193]
[594.99999416] [415.5953002]
[760.0000056] [1116.36068535]
[1179.00003028] [1118.27605009]
[1789.9999361] [1760.29

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [970.19888353]
[628.00000334] [698.19931316]
[442.00000328] [729.6841116]
[2284.99997044] [2623.26518011]
[1040.99999189] [826.43090868]
[809.99999654] [1315.28507376]
[3049.99987316] [3404.04675674]
[720.00000596] [654.4988451]
[877.99999249] [764.37413836]
[1292.00000429] [1605.99689913]
[7056.00004578] [4954.58460808]
[4176.99999142] [5754.38877869]
[13701.00020599] [14750.20832062]
[2948.0000515] [4029.62012291]
[16732.99985504] [14691.84033966]
[1815.00000048] [2032.77224493]
[16220.00016022] [12090.61985016]
[5967.00019073] [4746.90223885]
[473.00000042] [770.94775724]
[1078.9999795] [1111.89687538]
[452.00000319] [583.43866158]
[801.99998283] [831.34410381]
[2458.99994993] [2552.06222963]
[2492.99998236] [3290.59498596]
[582.00000203] [639.14381838]
[1405.9999671] [1399.06061125]
[8715.99991035] [11647.23310089]
[7634.00004578] [6518.961483]
[506.99999839] [636.49532604]
[665.99999094] [716.85272503]
[1815.00000048] [1527.4786582]
[707.00001383] [509.6461997]
[599

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1386.175056317977


## Conclusão



Como último experimento, vimos uma forma diferente de treinar a rede que trabalhamos tanto para terminar. Aqui estamos apenas vendo uma informação extra, que pode ser muito útil, uma vez que a forma que treinamos a rede MLP no experimento anterior é muito manual, já que tivemos que escrever todo o processo do cálculo do método. Entretanto, existe uma forma de otimizar esse processo utilizando um módulo de aprendizado de máquina especializado para <i>'deep learning'</i>, chamado `pytorch`. Nele há tudo de necessário para o treinamento de uma rede neural artificial, precisamos apenas do ter um dataset, como o que trabalhamos nesse experimento, e definir uma classe para armazenar as informações da nossa rede. Tal classe deve ser criada com base em uma classe própria do `pytorch`, a `nn.Module`.

Deste modo, precisamos seguir algumas regras para que o pytorch funcione, o que inclui, dentro do método `__init__`, definir as camadas da rede a partir do `nn.Sequential`, que é responsável por definir e organizar as camadas de neurônios uma por cima da outra. Desta forma, conseguimos implementar o método `forward`, o qual executa a rede do pytorch, computando os Tensores de saída dos Tensores de entrada, de maneira análoga à função `__call__` usada sem o pytorch. Tensores são objetos do pytorch que baseiam tudo dele, tal qual a classe `Valor`, armazenando dados e operações para o cálculo de gradiente local.

Além disso, o módulo também já tem implementadas várias funções de perda, das quais usamos a que computa o erro quadrático, o mesmo calculado pelo código escrito por nós no experimento anterior, dada por `nn.Module`. Precisamos apenas definir o otimizador que irá atualizar os parâmetros da rede.

Finalmente treinando nossa rede, foi compreendido que quando uma rede é criada ela já se encontra no que chamamos de modo de treino, porém podemos deixar claro que estamos trabalhando nesse modo a partir do comando `.train`. Também podemos explicitar que queremos trabalhar com os dados de avaliação, ou teste, com `.eval`, quando terminamos de treinar a nossa rede. Dessa forma, pudemos enfim usar o erro quadrático médio (RMSE) para medir a performance da nossa rede.

## Playground

